# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = 'D:/GitHub/2nd-ML100Days/data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [29]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
df['Cabin'] = df['Cabin'].fillna('None')
mean_df = df.groupby(['Cabin'])['Fare'].mean().reset_index()
mode_df = df.groupby(['Cabin'])['Fare'].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby(['Cabin'])['Fare'].median().reset_index()
max_df = df.groupby(['Cabin'])['Fare'].max().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=['Cabin'])
temp = pd.merge(temp, median_df, how='left', on=['Cabin'])
temp = pd.merge(temp, max_df, how='left', on=['Cabin'])
temp.columns = ['Cabin', 'Fare_Cabin_Mean', 'Fare_Cabin_Mode', 'Fare_Cabin_Median','Fare_Cabin_Max']

In [31]:
temp.head()

,Cabin,Fare_Cabin_Mean,Fare_Cabin_Mode,Fare_Cabin_Median,Fare_Cabin_Max
0,A10,40.1250,40.1250,40.1250,40.1250
1,A14,52.0000,52.0000,52.0000,52.0000
2,A16,39.6000,39.6000,39.6000,39.6000
3,A19,26.0000,26.0000,26.0000,26.0000
4,A20,56.9292,56.9292,56.9292,56.9292


In [32]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [34]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MMEncoder.fit_transform(df)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7038635542329971

In [44]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
df['Cabin'] = temp['Cabin']
df_add = df.merge( temp, how='left', on=['Cabin'])
df_add.drop(['Cabin'],axis=1,inplace=True)
df_add.fillna(-1,inplace=True)
train_X = MMEncoder.fit_transform(df_add)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7027526547217928

In [45]:
df_add

,Pclass,Age,SibSp,Parch,Fare,Fare_Cabin_Mean,Fare_Cabin_Mode,Fare_Cabin_Median,Fare_Cabin_Max
0,3,22.0,1,0,7.2500,40.1250,40.1250,40.1250,40.1250
1,1,38.0,1,0,71.2833,52.0000,52.0000,52.0000,52.0000
2,3,26.0,0,0,7.9250,39.6000,39.6000,39.6000,39.6000
3,1,35.0,1,0,53.1000,26.0000,26.0000,26.0000,26.0000
4,3,35.0,0,0,8.0500,56.9292,56.9292,56.9292,56.9292
5,3,-1.0,0,0,8.4583,30.0000,30.0000,30.0000,30.0000
6,1,54.0,0,0,51.8625,50.4958,50.4958,50.4958,50.4958
7,3,2.0,3,1,21.0750,35.5000,35.5000,35.5000,35.5000
8,3,27.0,0,2,11.1333,31.0000,31.0000,31.0000,31.0000
9,2,14.0,1,0,30.0708,50.0000,50.0000,50.0000,50.0000
